In [55]:
import pandas as pd
import experiment.preprocess as preprocess
from candidates_generation import triple_gen
import importlib
import numpy as np

In [71]:
importlib.reload(preprocess)

<module 'data_reader.preprocess' from '/Users/fieng/Project/KGC_RAG/data_reader/preprocess.py'>

In [118]:
path = 'data/economic/data.csv'
df = pd.read_csv(path)

In [188]:
path = 'data/economic/data.csv'
df = pd.read_csv(path)

pattern = r'\(:([a-zA-Z_]+)|_([a-zA-Z_]+)'

# Extract head
extracted_heads = df['n'].str.extract(pattern, expand=True)
extracted_heads = extracted_heads[0].fillna(extracted_heads[1])

# Extract tail 
extracted_tails = df['m'].str.extract(pattern, expand=True)
extracted_tails = extracted_tails[0].fillna(extracted_tails[1])

# Extract relation
extracted_rel = df['r'].str.extract(pattern, expand=True)
extracted_rel = extracted_rel[0].fillna(extracted_rel[1])

# Concatenate all incorrect index
incorrect_index = []
incorrect_index.append(df['m'][extracted_tails.isna() == True].index.tolist())
incorrect_index.append(df['n'][extracted_heads.isna() == True].index.tolist())
incorrect_index.append(df['r'][extracted_rel.isna() == True].index.tolist())
incorrect_index = [item for sublist in incorrect_index for item in sublist]

df = df[~df.index.isin(incorrect_index)]

In [189]:
# Extract head
extracted_heads = df['n'].str.extract(pattern, expand=True)
extracted_heads = extracted_heads[0].fillna(extracted_heads[1])

# Extract tail 
extracted_tails = df['m'].str.extract(pattern, expand=True)
extracted_tails = extracted_tails[0].fillna(extracted_tails[1])

# Extract relation
extracted_rel = df['r'].str.extract(pattern, expand=True)
extracted_rel = extracted_rel[0].fillna(extracted_rel[1])

In [196]:
new_df = pd.concat([extracted_heads,extracted_rel, extracted_tails],axis=1)
new_df.columns = ['Head','Relation','Tail']
df = new_df

In [217]:
type(df["Head"][0])

str

In [218]:
test_df = pd.read_csv('data/freebase/data_sample.csv')

In [219]:
test_df

,Head,Relation,Tail
0,Louise Fletcher,award,Academy Award for Best Actress
1,Island Records,music,Annie Lennox
2,The Sound of Music,film,Richard Rodgers
3,Ted Kennedy,government,104th United States Congress
4,West Virginia,location,Churches of Christ
...,...,...,...
62018,Deuce Bigalow: European Gigolo,award,Golden Raspberry Award for Worst Actor
62019,Charles Boyer,film,Around the World in 80 Days
62020,Vassar College,education,Merryl Streep
62021,Frasier,tv,Kelsey Grammer


In [72]:
path = 'data/codex/data.csv'
df = pd.read_csv(path)
evaluation_df, candidates_df, missing_df = preprocess.create_experiment_df(path)


/Users/fieng/Project/KGC_RAG/data_reader/preprocess.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assert nb_false_cand < 1, f'This program need a positive number of false candidates, you chose f{nb_false_cand}.'
/Users/fieng/Project/KGC_RAG/data_reader/preprocess.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # get the testing data


In [ ]:
from candidates_filtering.embedding import train_model
from candidates_filtering.embedding.get_emb_transe import get_list_dist

train_df = train_model.create_dataset(
    df)
test_df = train_model.create_dataset(
    df.sample(n=50))


embedding_dim = 5
model_kwargs = {"embedding_dim": embedding_dim}

model_dict = {}
#model_list = ['TransE','TransH','TransF','TransR','TransD']
model_list = ['TransE']
for model_name in model_list:
    experiment_name = model_name+f"_dim{embedding_dim}"
    model = train_model.create_pipeline(train_df, test_df,
                        model_name, model_kwargs, experiment_name)
    model_dict[model_name] = model

In [91]:
from candidates_filtering import triple_filter
model = model_dict['TransE']
list_dist = get_list_dist(candidates_df, model.model, train_df)
candidates_df['distance'] = list_dist

_, filtred_df = triple_filter.filter_best_threshold(model, candidates_df, missing_df, train_df)


In [90]:
import candidates_filtering
importlib.reload(candidates_filtering)

<module 'candidates_filtering' (<_frozen_importlib_external._NamespaceLoader object at 0x15eb2ee30>)>

In [106]:
import experiment


In [105]:
import experiment.filtering as filtering
importlib.reload(filtering)

<module 'experiment.filtering' from '/Users/fieng/Project/KGC_RAG/experiment/filtering.py'>

In [107]:
filtred_df = filtering.get_filtred_df(model, candidates_df,missing_df,train_df)

In [108]:
filtred_df

,Head,Relation,Tail
0,Telly Savalas,occupation,film editor
1,Telly Savalas,occupation,film director
3,Telly Savalas,occupation,film producer
6,Telly Savalas,occupation,stage actor
8,Telly Savalas,occupation,musician
...,...,...,...
1231422,Walt Disney Records,country,Germany
1231429,Richard Wagner,influenced by,Albert Einstein
1231452,Wassily Kandinsky,influenced by,Thomas Mann
1231453,Wassily Kandinsky,sibling,Erika Mann


In [165]:
import sys

In [166]:
print(sys.version)

3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:41:52) [Clang 15.0.7 ]


In [201]:
path = 'data/codex/data.csv'
df = pd.read_csv(path)

In [205]:
df[df['Head'] == 'Telly Savalas']

,Head,Relation,Tail
3339,Telly Savalas,"languages spoken, written, or signed",English
6700,Telly Savalas,occupation,television director
6785,Telly Savalas,occupation,radio personality
8486,Telly Savalas,educated at,Columbia University
8803,Telly Savalas,occupation,stage actor
15801,Telly Savalas,occupation,director
16106,Telly Savalas,occupation,character actor
17460,Telly Savalas,occupation,actor
23289,Telly Savalas,occupation,screenwriter
25954,Telly Savalas,occupation,television actor


In [208]:
candidates_df[['Head','Relation','Tail']].isin(df)

,Head,Relation,Tail
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
1231452,False,False,False
1231453,False,False,False
1231454,False,False,False
1231455,False,False,False


In [209]:
common = candidates_df.merge(df, on=['Head','Relation','Tail'], how='inner')

In [213]:
candidates_df[['Head','Relation','Tail']].to_csv('data/codex/candidates.csv',index=False)